# SQL tables to be included in the Postgres Docker image

In [1]:
%%file people_vaccination.sql
create table people_vaccination (
	id BIGSERIAL NOT NULL PRIMARY KEY,
	NAME VARCHAR(100) NOT NULL,
	LOCATION VARCHAR(100) NOT NULL,
	AGE SMALLINT NOT NULL CHECK (AGE > 0),
	IS_VACCINATED BOOLEAN NOT NULL
);
insert into people_vaccination (NAME, LOCATION, AGE, IS_VACCINATED) values ('Alex', 'Lviv', '45', 'TRUE');
insert into people_vaccination (NAME, LOCATION, AGE, IS_VACCINATED) values ('Taras', 'Lviv', '52', 'FALSE');
insert into people_vaccination (NAME, LOCATION, AGE, IS_VACCINATED) values ('Anna', 'Rivne', '22', 'FALSE');
insert into people_vaccination (NAME, LOCATION, AGE, IS_VACCINATED) values ('Iryna', 'Odessa', '35', 'TRUE');

Overwriting people_vaccination.sql


In [2]:
%%file relatives_relations.sql
create table relatives_relations (
	id BIGSERIAL NOT NULL PRIMARY KEY,
	ID_people_vaccinated BIGINT NOT NULL,
	NAME VARCHAR(100) NOT NULL,
	COUNTRY VARCHAR(100) NOT NULL,
	IS_VACCINATED BOOLEAN NOT NULL
);
insert into relatives_relations (ID_people_vaccinated, NAME, COUNTRY, IS_VACCINATED) values ('2', 'Relative1', 'Ukraine', 'FALSE');
insert into relatives_relations (ID_people_vaccinated, NAME, COUNTRY, IS_VACCINATED) values ('2', 'Relative2', 'Ukraine', 'FALSE');
insert into relatives_relations (ID_people_vaccinated, NAME, COUNTRY, IS_VACCINATED) values ('3', 'Relative3', 'Ukraine', 'TRUE');
insert into relatives_relations (ID_people_vaccinated, NAME, COUNTRY, IS_VACCINATED) values ('1', 'Relative4', 'Poland', 'FALSE');

Overwriting relatives_relations.sql


# Create a Postgres Docker image

In [3]:
from getpass import getpass

postgresql_password = getpass()

········


In [4]:
%%file Dockerfile
FROM postgres:9.6
ENV POSTGRES_USER postgres
ENV POSTGRES_PASSWORD postgresql_password
ENV POSTGRES_DB vaccinated_people
ADD people_vaccination.sql /docker-entrypoint-initdb.d/
ADD relatives_relations.sql /docker-entrypoint-initdb.d/
RUN apt update
RUN apt install sudo


Overwriting Dockerfile


> Build:

```sudo docker build -t postgres-example .```

> Run:

```sudo docker run -d postgres-example```

> Container's bash:

```sudo docker exec -it ``` container name or container id ``` bash```

```sudo docker exec -it id bash```

> psql:

```sudo -u postgres -i```

```psql```

```\list```

```\c vaccinated_people```

```\dt```

> SQL queries:

```SELECT * FROM people_vaccination;```

```SELECT * FROM relatives_relations;```

> JOIN with relatives

```SELECT 
    people_vaccination.name,
    people_vaccination.location,
    people_vaccination.age,
    people_vaccination.is_vaccinated,
    relatives_relations.name,
    relatives_relations.country,
    relatives_relations.is_vaccinated
FROM people_vaccination 
INNER JOIN relatives_relations 
ON people_vaccination.ID = id_people_vaccinated;```

> Logs:

```sudo docker logs ``` container_name or container_id

# Use Python to query tables

> Restart container with:

``` 
sudo docker ps
sudo docker stop RUNNING_CONTAINER_NAME_OR_ID
sudo docker run -e POSTGRES_PASSWORD=PASSWORD -d -p 5433:5432 postgres-example```

In [ ]:
!pip install tabulate

In [5]:
# for prettier tables output
from tabulate import tabulate

In [6]:
import psycopg2

conn = psycopg2.connect(f"host=localhost dbname=vaccinated_people user=postgres password={postgresql_password} port=5433")

In [7]:
# for convenience of handling long queries
import ipywidgets as widgets
from ipywidgets import Layout

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

query = 'SELECT \n    people_vaccination.name,\n    people_vaccination.location,\n    people_vaccination.age,\n    people_vaccination.is_vaccinated,\n    relatives_relations.name,\n    relatives_relations.country,\n    relatives_relations.is_vaccinated\nFROM people_vaccination \nJOIN relatives_relations \nON people_vaccination.ID = id_people_vaccinated;'

query_box = widgets.Textarea(value=query, placeholder='SQL query', description='SQL query', disabled=False, layout=Layout(width='100%', height='300px'), continuous_update = True)

display(query_box)

Textarea(value='SELECT \n    people_vaccination.name,\n    people_vaccination.location,\n    people_vaccinatio…

In [8]:
with conn:
    with conn.cursor() as curs:
        curs.execute(query_box.value)
        rows = curs.fetchall()
        
        for row in rows:
            print(row)


('Alex', 'Lviv', 45, True, 'Relative4', 'Poland', False)
('Taras', 'Lviv', 52, False, 'Relative2', 'Ukraine', False)
('Taras', 'Lviv', 52, False, 'Relative1', 'Ukraine', False)
('Anna', 'Rivne', 22, False, 'Relative3', 'Ukraine', True)


In [9]:
table1Name = "people_vaccination"
table2Name = "relatives_relations"

def print_table_content(tableName, max_items_to_print=10):
    with conn:
        with conn.cursor() as curs:
            curs.execute(f"SELECT column_name FROM INFORMATION_SCHEMA. COLUMNS WHERE TABLE_NAME = '{tableName}'")
            column_names = curs.fetchall()

            column_names = [column_name[0] for column_name in column_names]

            curs.execute(f"SELECT * FROM {tableName}")
            rows = curs.fetchall()
            print(f'{"-"*30}"{tableName}"{"-"*30}')
            print(tabulate(rows[:max_items_to_print], headers=column_names), end='\n'*2)
            
print_table_content(table1Name)
print_table_content(table2Name)

------------------------------"people_vaccination"------------------------------
  id  name    location      age  is_vaccinated
----  ------  ----------  -----  ---------------
   1  Alex    Lviv           45  True
   2  Taras   Lviv           52  False
   3  Anna    Rivne          22  False
   4  Iryna   Odessa         35  True

------------------------------"relatives_relations"------------------------------
  id    id_people_vaccinated  name       country    is_vaccinated
----  ----------------------  ---------  ---------  ---------------
   1                       2  Relative1  Ukraine    False
   2                       2  Relative2  Ukraine    False
   3                       3  Relative3  Ukraine    True
   4                       1  Relative4  Poland     False



In [10]:
conn.close()